In [1]:
import csv
import requests
from bs4 import BeautifulSoup

In [2]:

# Function to scrape and extract movie details
def scrape_movies(url):
    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all movie containers
    movie_containers = soup.find_all('div', class_='lister-item mode-advanced')

    movies = []
    for container in movie_containers:
        movie = {}

        # Movie name
        movie['Movie Name'] = container.h3.a.text

        # Directors (multiple directors handled)
        director_tags = container.select("div.lister-item-content p:nth-of-type(3) a")
        movie['Directors'] = [director.text for director in director_tags]

        # Duration
        duration = container.find('span', class_='runtime')
        movie['Duration'] = duration.text if duration else None

        # Year
        year = container.find('span', class_='lister-item-year').text
        movie['Year'] = year.strip('()')

        # Ratings
        ratings = container.find('div', class_='inline-block ratings-imdb-rating')
        movie['Ratings'] = ratings.strong.text if ratings else None

        # Metascore
        metascore = container.find('span', class_='metascore')
        movie['Metascore'] = metascore.text.strip() if metascore else None

        # Stars (multiple stars handled)
        star_tags = container.select("div.lister-item-content p:nth-of-type(4) a")
        movie['Stars'] = [star.text for star in star_tags]

        # Votes
        votes = container.find('span', attrs={'name': 'nv'})
        movie['Votes'] = votes['data-value'] if votes else None

        # Genre (multiple genres handled)
        genre_tags = container.select("div.lister-item-content p:nth-of-type(1) span.genre")
        movie['Genre'] = [genre.text.strip() for genre in genre_tags]

        # Gross Collection
        gross = container.find('span', class_='text-muted text-small')
        movie['Gross Collection'] = gross.contents[2].strip() if gross else None

        # Popularity
        popularity = container.find('span', class_='popularityTrend')
        movie['Popularity'] = popularity['title'] if popularity else None

        # Certification
        certificate = container.find('span', class_='certificate')
        movie['Certification'] = certificate.text if certificate else None

        movies.append(movie)

    return movies


In [3]:

# Main function to scrape all pages until desired count is reached
def scrape_all_movies(url, desired_count):
    all_movies = []

    page = 1
    count = 0
    while count < desired_count:
        print(f"Scraping page {page}...")
        page_url = url + f'&start={((page-1)*50)+1}'
        movies = scrape_movies(page_url)
        all_movies.extend(movies)
        count += len(movies)
        page += 1

    return all_movies[:desired_count]

# URL for scraping
url = 'https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1'

# Desired count of movies
desired_count = 1743

# Scrape all movie details
all_movies = scrape_all_movies(url, desired_count)

# Define the field names for CSV files
csv_fields1 = ['Sno', 'Movie Name', 'Director1', 'Director2', 'Duration', 'Year', 'Ratings', 'Metascore']
csv_fields2 = ['Movie Name', 'Star1', 'Star2', 'Star3', 'Star4', 'Votes', 'Genre1', 'Genre2', 'Genre3', 'Gross Collection', 'Popularity', 'Certification']

# Generate CSV file 1
with open('movies_data_1.csv', mode='w', encoding='utf-8', newline='') as file1:
    writer1 = csv.DictWriter(file1, fieldnames=csv_fields1)
    writer1.writeheader()
    for index, movie in enumerate(all_movies):
        writer1.writerow({'Sno': index+1,
                          'Movie Name': movie['Movie Name'],
                          'Director1': movie['Directors'][0] if movie['Directors'] else None,
                          'Director2': movie['Directors'][1] if len(movie['Directors']) > 1 else None,
                          'Duration': movie['Duration'],
                          'Year': movie['Year'],
                          'Ratings': movie['Ratings'],
                          'Metascore': movie['Metascore']})

# Generate CSV file 2
with open('movies_data_2.csv', mode='w', encoding='utf-8', newline='') as file2:
    writer2 = csv.DictWriter(file2, fieldnames=csv_fields2)
    writer2.writeheader()
    for movie in all_movies:
        writer2.writerow({'Movie Name': movie['Movie Name'],
                          'Star1': movie['Stars'][0] if movie['Stars'] else None,
                          'Star2': movie['Stars'][1] if len(movie['Stars']) > 1 else None,
                          'Star3': movie['Stars'][2] if len(movie['Stars']) > 2 else None,
                          'Star4': movie['Stars'][3] if len(movie['Stars']) > 3 else None,
                          'Votes': movie['Votes'],
                          'Genre1': movie['Genre'][0] if movie['Genre'] else None,
                          'Genre2': movie['Genre'][1] if len(movie['Genre']) > 1 else None,
                          'Genre3': movie['Genre'][2] if len(movie['Genre']) > 2 else None,
                          'Gross Collection': movie['Gross Collection'],
                          'Popularity': movie['Popularity'],
                          'Certification': movie['Certification']})

print("Scraping and CSV generation completed successfully!")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping and CSV generation completed successfully!


In [4]:
import csv

# Function to print data from CSV file
def print_csv_data(filename):
    with open(filename, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            print(row)

# Print data from movies_data_1.csv
print("Data from movies_data_1.csv:")
print_csv_data('movies_data_1.csv')

Data from movies_data_1.csv:
['Sno', 'Movie Name', 'Director1', 'Director2', 'Duration', 'Year', 'Ratings', 'Metascore']
['1', 'The Dark Knight', 'Christopher Nolan', 'Christian Bale', '152 min', '2008', '9.0', '84']
['2', 'The Lord of the Rings: The Return of the King', 'Peter Jackson', 'Elijah Wood', '201 min', '2003', '9.0', '94']
['3', 'Inception', 'Christopher Nolan', 'Leonardo DiCaprio', '148 min', '2010', '8.8', '74']
['4', 'The Lord of the Rings: The Fellowship of the Ring', 'Peter Jackson', 'Elijah Wood', '178 min', '2001', '8.8', '92']
['5', 'The Lord of the Rings: The Two Towers', 'Peter Jackson', 'Elijah Wood', '179 min', '2002', '8.8', '87']
['6', 'The Matrix', 'Lana Wachowski', 'Lilly Wachowski', '136 min', '1999', '8.7', '73']
['7', 'Star Wars: Episode V - The Empire Strikes Back', 'Irvin Kershner', 'Mark Hamill', '124 min', '1980', '8.7', '82']
['8', 'Soorarai Pottru', 'Sudha Kongara', 'Suriya', '153 min', '2020', '8.7', '']
['9', 'Star Wars', 'George Lucas', 'Mark Hami

In [5]:
import csv

# Function to print data from CSV file
def print_csv_data(filename):
    with open(filename, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            print(row)

# Print data from movies_data_2.csv
print("Data from movies_data_2.csv:")
print_csv_data('movies_data_2.csv')

Data from movies_data_2.csv:
['Movie Name', 'Star1', 'Star2', 'Star3', 'Star4', 'Votes', 'Genre1', 'Genre2', 'Genre3', 'Gross Collection', 'Popularity', 'Certification']
['The Dark Knight', '', '', '', '', '2718979', 'Action, Crime, Drama', '', '', '', '', 'UA']
['The Lord of the Rings: The Return of the King', '', '', '', '', '1887541', 'Action, Adventure, Drama', '', '', '', '', 'U']
['Inception', '', '', '', '', '2413690', 'Action, Adventure, Sci-Fi', '', '', '', '', 'UA']
['The Lord of the Rings: The Fellowship of the Ring', '', '', '', '', '1916169', 'Action, Adventure, Drama', '', '', '', '', 'U']
['The Lord of the Rings: The Two Towers', '', '', '', '', '1703685', 'Action, Adventure, Drama', '', '', '', '', 'UA']
['The Matrix', '', '', '', '', '1958341', 'Action, Sci-Fi', '', '', '', '', 'A']
['Star Wars: Episode V - The Empire Strikes Back', '', '', '', '', '1321424', 'Action, Adventure, Fantasy', '', '', '', '', 'UA']
['Soorarai Pottru', '', '', '', '', '119516', 'Action, Dram

In [9]:
import sqlite3
import csv

# Create a connection to the database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Create Table 1: Movies
cursor.execute('''CREATE TABLE IF NOT EXISTS Movies
                (Sno INTEGER, MovieName TEXT, Director1 TEXT, Director2 TEXT, Duration TEXT, Year TEXT, Ratings REAL, Metascore TEXT)''')

# Create Table 2: MovieDetails
cursor.execute('''CREATE TABLE IF NOT EXISTS MovieDetails
                (MovieName TEXT, Star1 TEXT, Star2 TEXT, Star3 TEXT, Star4 TEXT, Votes INTEGER, Genre1 TEXT, Genre2 TEXT, Genre3 TEXT, GrossCollection TEXT, Popularity TEXT, Certification TEXT)''')

# Function to insert data from CSV into table
def insert_data_from_csv(csv_file, table_name):
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        if table_name == 'Movies':
            rows = [(row['Sno'], row['Movie Name'], row['Director1'], row['Director2'], row['Duration'], row['Year'], row['Ratings'], row['Metascore'])
                    for row in reader]
        elif table_name == 'MovieDetails':
            rows = [(row['Movie Name'], row['Star1'], row['Star2'], row['Star3'], row['Star4'], row['Votes'], row['Genre1'], row['Genre2'], row['Genre3'], row['Gross Collection'], row['Popularity'], row['Certification'])
                    for row in reader]
    cursor.executemany(f'INSERT INTO {table_name} VALUES (?, ?, ?, ?, ?, ?, ?, ?)' if table_name == 'Movies' else
                     f'INSERT INTO {table_name} VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', rows)
    conn.commit()

# Insert data into Table 1: Movies
insert_data_from_csv('movies_data_1.csv', 'Movies')

# Insert data into Table 2: MovieDetails
insert_data_from_csv('movies_data_2.csv', 'MovieDetails')

# Close the database connection
conn.close()



In [10]:
import sqlite3

# Create a SQLite database connection
conn = sqlite3.connect('imdb_database.db')

# Query 1: Display all the details of movies created by directors Christopher and Matt Reeves.
query1 = '''SELECT * FROM Movies WHERE "Director Name" LIKE '%Christopher%' OR "Director Name" LIKE '%Matt Reeves%' '''
result1 = conn.execute(query1)
print("Movies created by Christopher and Matt Reeves:")
for row in result1:
    print(row)

# Query 2: Display all the details of movies with a duration of 140 minutes to 190 minutes.
query2 = '''SELECT * FROM Movies WHERE Duration BETWEEN 140 AND 190 '''
result2 = conn.execute(query2)
print("Movies with a duration of 140 minutes to 190 minutes:")
for row in result2:
    print(row)

# Query 3: Display all details of movies with ratings above 7 in ascending order.
query3 = '''SELECT * FROM Movies WHERE ratings > 7 ORDER BY ratings ASC'''
result3 = conn.execute(query3)
print("Movies with ratings above 7 in ascending order:")
for row in result3:
    print(row)

# Query 4: Display all movie names in descending order.
query4 = '''SELECT MovieName FROM Movies ORDER BY MovieName DESC'''
result4 = conn.execute(query4)
print("Movie names in descending order:")
for row in result4:
    print(row[0])

# Query 5: Display movie name starts with ‘P’ and their rating is greater than 7.
query5 = '''SELECT MovieName FROM Movies WHERE MovieName LIKE 'P%' AND ratings > 7'''
result5 = conn.execute(query5)
print("Movie names starting with 'P' and rating greater than 7:")
for row in result5:
    print(row[0])

# Query 6: Display all movie names with star Arnold Schwarzenegger in ascending order.
query6 = '''SELECT MovieName FROM MovieDetails WHERE stars LIKE '%Arnold Schwarzenegger%' ORDER BY MovieName ASC'''
result6 = conn.execute(query6)
print("Movie names with star Arnold Schwarzenegger in ascending order:")
for row in result6:
    print(row[0])

# Query 7: Display all details of the movie with the highest number of votes.
query7 = '''SELECT * FROM MovieDetails WHERE votes = (SELECT MAX(votes) FROM MovieDetails)'''
result7 = conn.execute(query7)
print("Details of the movie with the highest number of votes:")
for row in result7:
    print(row)

# Query 8: Display movie names with gross collections in descending order.
query8 = '''SELECT MovieName FROM MovieDetails ORDER BY "Gross collection" DESC'''
result8 = conn.execute(query8)
print("Movie names with gross collections in descending order:")
for row in result8:
    print(row[0])

# Query 9: Display the gross collection of movies with the star Arnold.
query9 = '''SELECT MovieName, "Gross collection" FROM MovieDetails WHERE stars LIKE '%Arnold%' '''
result9 = conn.execute(query9)
print("Gross collection of movies with the star Arnold:")
for row in result9:
    print(row[0], row[1])

# Query 10: Display all details of movies with comedy and action genres.
query10 = '''SELECT * FROM MovieDetails WHERE Genre LIKE '%comedy%' AND Genre LIKE '%action%' '''
result10 = conn.execute(query10)
print("Details of movies with comedy and action genres:")
for row in result10:
    print(row)

# Close the database connection
conn.close()


Movies created by Christopher and Matt Reeves:
Movies with a duration of 140 minutes to 190 minutes:
(1, 'The Dark Knight', 'Christopher Nolan', 'Christian Bale', '152 min', '2008', 9.0, 84.0)
(3, 'Inception', 'Christopher Nolan', 'Leonardo DiCaprio', '148 min', '2010', 8.8, 74.0)
(4, 'The Lord of the Rings: The Fellowship of the Ring', 'Peter Jackson', 'Elijah Wood', '178 min', '2001', 8.8, 92.0)
(5, 'The Lord of the Rings: The Two Towers', 'Peter Jackson', 'Elijah Wood', '179 min', '2002', 8.8, 87.0)
(8, 'Soorarai Pottru', 'Sudha Kongara', 'Suriya', '153 min', '2020', 8.7, None)
(13, 'Sita Ramam', 'Hanu Raghavapudi', 'Dulquer Salmaan', '163 min', '2022', 8.6, None)
(15, 'Gladiator', 'Ridley Scott', 'Russell Crowe', '155 min', '2000', 8.5, 67.0)
(16, 'Avengers: Endgame', 'Anthony Russo', 'Joe Russo', '181 min', '2019', 8.4, 78.0)
(19, 'Avengers: Infinity War', 'Anthony Russo', 'Joe Russo', '149 min', '2018', 8.4, 68.0)
(21, 'The Dark Knight Rises', 'Christopher Nolan', 'Christian Bale

OperationalError: no such column: MovieName

In [15]:
import csv

def print_csv_column_names(filename):
    with open(filename, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the first row as the header row
        print("Column names:")
        print(headers)

# Print column names of movies_data_1.csv
print_csv_column_names('movies_data_1.csv')

# Print column names of movies_data_2.csv
print_csv_column_names('movies_data_2.csv')


Column names:
['Sno', 'Movie Name', 'Director1', 'Director2', 'Duration', 'Year', 'Ratings', 'Metascore']
Column names:
['Movie Name', 'Star1', 'Star2', 'Star3', 'Star4', 'Votes', 'Genre1', 'Genre2', 'Genre3', 'Gross Collection', 'Popularity', 'Certification']


In [20]:
import sqlite3

# Create a SQLite database connection
conn = sqlite3.connect('imdb_database.db')

# Subquery 1: Display all details from both tables where movie names are the same.
subquery1 = '''
SELECT *
FROM Movies
INNER JOIN MovieDetails ON Movies."Movie Name" = MovieDetails."Movie Name"
'''
result1 = conn.execute(subquery1)
print("Details from both tables where movie names are the same:")
for row in result1:
    print(row)

# Subquery 2: Display all movie names, Directors, Ratings, and GrossCollection where the genre is action.
subquery2 = '''
SELECT Movies."Movie Name", Movies.Director1, Movies.Ratings, MovieDetails."Gross Collection"
FROM Movies
INNER JOIN MovieDetails ON Movies."Movie Name" = MovieDetails."Movie Name"
WHERE MovieDetails.Genre1 LIKE '%action%' OR MovieDetails.Genre2 LIKE '%action%' OR MovieDetails.Genre3 LIKE '%action%'
'''
result2 = conn.execute(subquery2)
print("Movie names, Directors, Ratings, and GrossCollection where the genre is action:")
for row in result2:
    print(row)

# Subquery 3: Display all details from both tables with the highest GrossCollection.
subquery3 = '''
SELECT *
FROM Movies
INNER JOIN MovieDetails ON Movies."Movie Name" = MovieDetails."Movie Name"
WHERE MovieDetails."Gross Collection" = (SELECT MAX("Gross Collection") FROM MovieDetails)
'''
result3 = conn.execute(subquery3)
print("Details from both tables with the highest GrossCollection:")
for row in result3:
    print(row)

# Subquery 4: Display all details from both tables with the highest Ratings.
subquery4 = '''
SELECT *
FROM Movies
INNER JOIN MovieDetails ON Movies."Movie Name" = MovieDetails."Movie Name"
WHERE Movies.Ratings = (SELECT MAX(Ratings) FROM Movies)
'''
result4 = conn.execute(subquery4)
print("Details from both tables with the highest Ratings:")
for row in result4:
    print(row)

# Subquery 5: Display all details from both tables with the lowest GrossCollection and lowest Ratings.
subquery5 = '''
SELECT *
FROM Movies
INNER JOIN MovieDetails ON Movies."Movie Name" = MovieDetails."Movie Name"
WHERE MovieDetails."Gross Collection" = (SELECT MIN("Gross Collection") FROM MovieDetails)
    AND Movies.Ratings = (SELECT MIN(Ratings) FROM Movies)
'''
result5 = conn.execute(subquery5)
print("Details from both tables with the lowest GrossCollection and lowest Ratings:")
for row in result5:
    print(row)

# Close the database connection
conn.close()


Details from both tables where movie names are the same:
(1, 'The Dark Knight', 'Christopher Nolan', 'Christian Bale', '152 min', '2008', 9.0, 84.0, 'The Dark Knight', None, None, None, None, 2714833, 'Action, Crime, Drama', None, None, None, None, 'UA')
(2, 'The Lord of the Rings: The Return of the King', 'Peter Jackson', 'Elijah Wood', '201 min', '2003', 9.0, 94.0, 'The Lord of the Rings: The Return of the King', None, None, None, None, 1885058, 'Action, Adventure, Drama', None, None, None, None, 'U')
(3, 'Inception', 'Christopher Nolan', 'Leonardo DiCaprio', '148 min', '2010', 8.8, 74.0, 'Inception', None, None, None, None, 2409932, 'Action, Adventure, Sci-Fi', None, None, None, None, 'UA')
(4, 'The Lord of the Rings: The Fellowship of the Ring', 'Peter Jackson', 'Elijah Wood', '178 min', '2001', 8.8, 92.0, 'The Lord of the Rings: The Fellowship of the Ring', None, None, None, None, 1913821, 'Action, Adventure, Drama', None, None, None, None, 'U')
(5, 'The Lord of the Rings: The Two

In [21]:
import pandas as pd

# Load movies_data_1.csv into a dataframe
df_movies = pd.read_csv('movies_data_1.csv')

# Load movies_data_2.csv into a dataframe
df_movie_details = pd.read_csv('movies_data_2.csv')




In [22]:
from pandasql import sqldf

# Example: Display all details from both tables where movie names are the same
query1 = '''
SELECT *
FROM df_movies m
INNER JOIN df_movie_details md ON m."Movie Name" = md."Movie Name"
'''

# Execute the query using sqldf from pandasql
result1 = sqldf(query1, globals())

# Print the result
print(result1)


       Sno                                         Movie Name  \
0        1                                    The Dark Knight   
1        2      The Lord of the Rings: The Return of the King   
2        3                                          Inception   
3        4  The Lord of the Rings: The Fellowship of the Ring   
4        5              The Lord of the Rings: The Two Towers   
...    ...                                                ...   
1800  1739                               Dragonball Evolution   
1801  1740                                  Battlefield Earth   
1802  1741                                        Heropanti 2   
1803  1742                                             Laxmii   
1804  1743                                  Alone in the Dark   

              Director1            Director2 Duration  Year  Ratings  \
0     Christopher Nolan       Christian Bale  152 min  2008      9.0   
1         Peter Jackson          Elijah Wood  201 min  2003      9.0   
2  

In [23]:
from pandasql import sqldf

# Query 1: Display all details from both tables where movie names are the same
query1 = '''
SELECT *
FROM df_movies m
INNER JOIN df_movie_details md ON m."Movie Name" = md."Movie Name"
'''

result1 = sqldf(query1, globals())
print("Query 1 Result:")
print(result1)
print()

# Query 2: Display all movie names, Director, ratings, and gross collection where the genre is action
query2 = '''
SELECT m."Movie Name", m.Directors, m.Ratings, md."Gross Collection"
FROM df_movies m
INNER JOIN df_movie_details md ON m."Movie Name" = md."Movie Name"
WHERE md.Genre LIKE '%action%'
'''

result2 = sqldf(query2, globals())
print("Query 2 Result:")
print(result2)
print()

# Query 3: Display all details from both tables with the highest gross collection
query3 = '''
SELECT *
FROM df_movies m
INNER JOIN df_movie_details md ON m."Movie Name" = md."Movie Name"
WHERE md."Gross Collection" = (SELECT MAX("Gross Collection") FROM df_movie_details)
'''

result3 = sqldf(query3, globals())
print("Query 3 Result:")
print(result3)
print()

# Query 4: Display all details from both tables with the highest ratings
query4 = '''
SELECT *
FROM df_movies m
INNER JOIN df_movie_details md ON m."Movie Name" = md."Movie Name"
WHERE m.Ratings = (SELECT MAX(Ratings) FROM df_movies)
'''

result4 = sqldf(query4, globals())
print("Query 4 Result:")
print(result4)
print()

# Query 5: Display all details from both tables with the lowest gross collection and lowest ratings
query5 = '''
SELECT *
FROM df_movies m
INNER JOIN df_movie_details md ON m."Movie Name" = md."Movie Name"
WHERE md."Gross Collection" = (SELECT MIN("Gross Collection") FROM df_movie_details)
    AND m.Ratings = (SELECT MIN(Ratings) FROM df_movies)
'''

result5 = sqldf(query5, globals())
print("Query 5 Result:")
print(result5)
print()


Query 1 Result:
       Sno                                         Movie Name  \
0        1                                    The Dark Knight   
1        2      The Lord of the Rings: The Return of the King   
2        3                                          Inception   
3        4  The Lord of the Rings: The Fellowship of the Ring   
4        5              The Lord of the Rings: The Two Towers   
...    ...                                                ...   
1800  1739                               Dragonball Evolution   
1801  1740                                  Battlefield Earth   
1802  1741                                        Heropanti 2   
1803  1742                                             Laxmii   
1804  1743                                  Alone in the Dark   

              Director1            Director2 Duration  Year  Ratings  \
0     Christopher Nolan       Christian Bale  152 min  2008      9.0   
1         Peter Jackson          Elijah Wood  201 min  2003

PandaSQLException: (sqlite3.OperationalError) no such column: m.Directors
[SQL: 
SELECT m."Movie Name", m.Directors, m.Ratings, md."Gross Collection"
FROM df_movies m
INNER JOIN df_movie_details md ON m."Movie Name" = md."Movie Name"
WHERE md.Genre LIKE '%action%'
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [24]:
import pandas as pd

# Load Table 1 data
df_movies = pd.read_csv('movies_data_1.csv')

# Load Table 2 data
df_movie_details = pd.read_csv('movies_data_2.csv')


In [25]:
# Print column details for Table 1 (df_movies)
print("Table 1 - df_movies:")
print(df_movies.columns)

# Print column details for Table 2 (df_movie_details)
print("Table 2 - df_movie_details:")
print(df_movie_details.columns)


Table 1 - df_movies:
Index(['Sno', 'Movie Name', 'Director1', 'Director2', 'Duration', 'Year',
       'Ratings', 'Metascore'],
      dtype='object')
Table 2 - df_movie_details:
Index(['Movie Name', 'Star1', 'Star2', 'Star3', 'Star4', 'Votes', 'Genre1',
       'Genre2', 'Genre3', 'Gross Collection', 'Popularity', 'Certification'],
      dtype='object')


In [26]:
from pandasql import sqldf

query1 = """
SELECT *
FROM df_movies
WHERE Director1 IN ('Christopher', 'Matt Reeves')
"""

result1 = sqldf(query1, globals())
print(result1)


   Sno                      Movie Name    Director1         Director2  \
0  130                      The Batman  Matt Reeves  Robert Pattinson   
1  223  Dawn of the Planet of the Apes  Matt Reeves       Gary Oldman   
2  305  War for the Planet of the Apes  Matt Reeves       Andy Serkis   
3  503                     Cloverfield  Matt Reeves        Mike Vogel   

  Duration  Year  Ratings  Metascore  
0  176 min  2022      7.8       72.0  
1  130 min  2014      7.6       79.0  
2  140 min  2017      7.4       82.0  
3   85 min  2008      7.0       64.0  


In [27]:
query2 = """
SELECT *
FROM df_movies
WHERE Duration BETWEEN 140 AND 190
"""

result2 = sqldf(query2, globals())
print(result2)


      Sno                                         Movie Name  \
0       1                                    The Dark Knight   
1       3                                          Inception   
2       4  The Lord of the Rings: The Fellowship of the Ring   
3       5              The Lord of the Rings: The Two Towers   
4       8                                    Soorarai Pottru   
..    ...                                                ...   
196  1690                                           Shehzada   
197  1737                                              Liger   
198  1738                                             Gunday   
199  1741                                        Heropanti 2   
200  1742                                             Laxmii   

             Director1            Director2 Duration  Year  Ratings  Metascore  
0    Christopher Nolan       Christian Bale  152 min  2008      9.0       84.0  
1    Christopher Nolan    Leonardo DiCaprio  148 min  2010      8.8  

In [28]:
query3 = """
SELECT *
FROM df_movies
WHERE ratings > 7
ORDER BY ratings ASC
"""

result3 = sqldf(query3, globals())
print(result3)


     Sno                                         Movie Name  \
0    430                                   Fast & Furious 7   
1    431                                          The Mummy   
2    432                                Mission: Impossible   
3    433                                           Free Guy   
4    434                                       Wrath of Man   
..   ...                                                ...   
482    3                                          Inception   
483    4  The Lord of the Rings: The Fellowship of the Ring   
484    5              The Lord of the Rings: The Two Towers   
485    1                                    The Dark Knight   
486    2      The Lord of the Rings: The Return of the King   

             Director1          Director2 Duration  Year  Ratings  Metascore  
0            James Wan         Vin Diesel  137 min  2015      7.1       67.0  
1      Stephen Sommers     Brendan Fraser  124 min  1999      7.1       48.0  
2     

In [29]:
query4 = """
SELECT 'Movie Name'
FROM df_movies
ORDER BY 'Movie Name' DESC
"""

result4 = sqldf(query4, globals())
print(result4)


     'Movie Name'
0      Movie Name
1      Movie Name
2      Movie Name
3      Movie Name
4      Movie Name
...           ...
1738   Movie Name
1739   Movie Name
1740   Movie Name
1741   Movie Name
1742   Movie Name

[1743 rows x 1 columns]


In [36]:
import pandas as pd
from pandasql import sqldf

# Assuming you have loaded the data into a DataFrame: df_movies

# Define the function to execute SQL queries
def pysqldf(query):
    return sqldf(query, globals())

# Subquery 5: Display movie name and ratings where the movie name starts with 'P' and ratings are greater than 7
query5 = """
SELECT "Movie Name", ratings
FROM df_movies
WHERE "Movie Name" LIKE 'P%' AND ratings > 7
"""

# Execute the query using the pysqldf function
result5 = pysqldf(query5)
print(result5)


                                           Movie Name  Ratings
0                                    Paan Singh Tomar      8.2
1   Pirates of the Caribbean: The Curse of the Bla...      8.1
2                             Per un pugno di dollari      7.9
3                                            Predator      7.8
4                                  Ponniyin Selvan: I      7.7
5                           Pushpa: The Rise - Part 1      7.6
6                                      Predestination      7.4
7          Pirates of the Caribbean: Dead Man's Chest      7.3
8                                        Patriots Day      7.3
9                                         Point Break      7.2
10                                               Prey      7.1
11           Pirates of the Caribbean: At World's End      7.1
12                                      Planet Terror      7.1
13                                            Payback      7.1


In [37]:
# Subquery: Display all details from both tables where movie names are the same
subquery6 = """
SELECT *
FROM df_movies
INNER JOIN df_movie_details ON df_movies.`Movie Name` = df_movie_details.`Movie Name`
"""
result6 = pysqldf(subquery6)
print(result6)


       Sno                                         Movie Name  \
0        1                                    The Dark Knight   
1        2      The Lord of the Rings: The Return of the King   
2        3                                          Inception   
3        4  The Lord of the Rings: The Fellowship of the Ring   
4        5              The Lord of the Rings: The Two Towers   
...    ...                                                ...   
1800  1739                               Dragonball Evolution   
1801  1740                                  Battlefield Earth   
1802  1741                                        Heropanti 2   
1803  1742                                             Laxmii   
1804  1743                                  Alone in the Dark   

              Director1            Director2 Duration  Year  Ratings  \
0     Christopher Nolan       Christian Bale  152 min  2008      9.0   
1         Peter Jackson          Elijah Wood  201 min  2003      9.0   
2  

In [38]:
subquery7 = """
SELECT df_movies.`Movie Name`, df_movies.`Director2`, df_movies.Ratings, df_movie_details.`Gross Collection`
FROM df_movies
INNER JOIN df_movie_details ON df_movies.`Movie Name` = df_movie_details.`Movie Name`
WHERE df_movie_details.Genre1 = 'action'
"""
result7 = pysqldf(subquery7)
print(result7)



Empty DataFrame
Columns: [Movie Name, Director2, Ratings, Gross Collection]
Index: []


In [39]:
# Subquery: Display all details from both tables with the highest gross collection
subquery8 = """
SELECT *
FROM df_movies
INNER JOIN df_movie_details ON df_movies.`Movie Name` = df_movie_details.`Movie Name`
WHERE df_movie_details.`Gross collection` = (
    SELECT MAX(`Gross collection`)
    FROM df_movie_details
)
"""
result8 = pysqldf(subquery8)
print(result8)


Empty DataFrame
Columns: [Sno, Movie Name, Director1, Director2, Duration, Year, Ratings, Metascore, Movie Name, Star1, Star2, Star3, Star4, Votes, Genre1, Genre2, Genre3, Gross Collection, Popularity, Certification]
Index: []


In [40]:
# Subquery: Display all details from both tables with the highest ratings
subquery9 = """
SELECT *
FROM df_movies
INNER JOIN df_movie_details ON df_movies.`Movie Name` = df_movie_details.`Movie Name`
WHERE df_movies.ratings = (
    SELECT MAX(ratings)
    FROM df_movies
)
"""
result9 = pysqldf(subquery9)
print(result9)


   Sno                                     Movie Name          Director1  \
0    1                                The Dark Knight  Christopher Nolan   
1    2  The Lord of the Rings: The Return of the King      Peter Jackson   

        Director2 Duration  Year  Ratings  Metascore  \
0  Christian Bale  152 min  2008      9.0       84.0   
1     Elijah Wood  201 min  2003      9.0       94.0   

                                      Movie Name Star1 Star2 Star3 Star4  \
0                                The Dark Knight  None  None  None  None   
1  The Lord of the Rings: The Return of the King  None  None  None  None   

     Votes                    Genre1 Genre2 Genre3 Gross Collection  \
0  2718979      Action, Crime, Drama   None   None             None   
1  1887541  Action, Adventure, Drama   None   None             None   

  Popularity Certification  
0       None            UA  
1       None             U  


In [41]:
# Subquery: Display all details from both tables with the lowest gross collection and lowest ratings
subquery10 = """
SELECT *
FROM df_movies
INNER JOIN df_movie_details ON df_movies.`Movie Name` = df_movie_details.`Movie Name`
WHERE df_movie_details.`Gross collection` = (
    SELECT MIN(`Gross collection`)
    FROM df_movie_details
) AND df_movies.ratings = (
    SELECT MIN(ratings)
    FROM df_movies
)
"""
result10 = pysqldf(subquery10)
print(result10)


Empty DataFrame
Columns: [Sno, Movie Name, Director1, Director2, Duration, Year, Ratings, Metascore, Movie Name, Star1, Star2, Star3, Star4, Votes, Genre1, Genre2, Genre3, Gross Collection, Popularity, Certification]
Index: []


In [42]:
# Subquery: Display all movie names with star Arnold Schwarzenegger in ascending order
subquery11 = """
SELECT `Movie Name`
FROM df_movie_details
WHERE Star1 = 'Arnold Schwarzenegger'
ORDER BY `Movie Name` ASC
"""
result11 = pysqldf(subquery11)
print(result11)



Empty DataFrame
Columns: [Movie Name]
Index: []


In [43]:
# Subquery: Display all details of the movie with the highest number of votes
subquery12 = """
SELECT *
FROM df_movie_details
WHERE votes = (
    SELECT MAX(votes)
    FROM df_movie_details
)
"""
result12 = pysqldf(subquery12)
print(result12)


        Movie Name Star1 Star2 Star3 Star4    Votes                Genre1  \
0  The Dark Knight  None  None  None  None  2718979  Action, Crime, Drama   

  Genre2 Genre3 Gross Collection Popularity Certification  
0   None   None             None       None            UA  


In [44]:
# Subquery: Display movie names with gross collections in descending order
subquery13 = """
SELECT `Movie Name`
FROM df_movie_details
ORDER BY `Gross collection` DESC
"""
result13 = pysqldf(subquery13)
print(result13)


                                             Movie Name
0                                       The Dark Knight
1         The Lord of the Rings: The Return of the King
2                                             Inception
3     The Lord of the Rings: The Fellowship of the Ring
4                 The Lord of the Rings: The Two Towers
...                                                 ...
1738                               Dragonball Evolution
1739                                  Battlefield Earth
1740                                        Heropanti 2
1741                                             Laxmii
1742                                  Alone in the Dark

[1743 rows x 1 columns]


In [45]:
# Subquery: Display the gross collection of movies with the star Arnold
subquery14 = """
SELECT df_movies.`Movie Name`, df_movie_details.`Gross Collection`
FROM df_movies
INNER JOIN df_movie_details ON df_movies.`Movie Name` = df_movie_details.`Movie Name`
WHERE df_movie_details.`Star1` = 'Arnold Schwarzenegger' OR
      df_movie_details.`Star2` = 'Arnold Schwarzenegger' OR
      df_movie_details.`Star3` = 'Arnold Schwarzenegger' OR
      df_movie_details.`Star4` = 'Arnold Schwarzenegger'
"""
result14 = pysqldf(subquery14)
print(result14)



Empty DataFrame
Columns: [Movie Name, Gross Collection]
Index: []


In [46]:
subquery15 = """
SELECT *
FROM df_movies
INNER JOIN df_movie_details ON df_movies.`Movie Name` = df_movie_details.`Movie Name`
WHERE df_movie_details.`Genre1` IN ('comedy', 'action') OR df_movie_details.`Genre2` IN ('comedy', 'action') OR df_movie_details.`Genre3` IN ('comedy', 'action')
"""
result15 = pysqldf(subquery15)
print(result15)






Empty DataFrame
Columns: [Sno, Movie Name, Director1, Director2, Duration, Year, Ratings, Metascore, Movie Name, Star1, Star2, Star3, Star4, Votes, Genre1, Genre2, Genre3, Gross Collection, Popularity, Certification]
Index: []
